In [18]:
import os
import requests
import io
import time
from PIL import Image
import hashlib
from selenium import webdriver
DRIVER_PATH='C:/Users/Smurf/Desktop/chromedriver.exe'

In [19]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

      
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Gefunden: {number_results} Suchergebnisse. Extrahiere Links von {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
          
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

              
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Gefundene Bilderlinks: {len(image_urls)}, Fertig!")
                break
        else:
            print("Gefundene Bilderlinks:", len(image_urls), ", suche weiter ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

       
        results_start = len(thumbnail_results)

    return image_urls

In [20]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Konnte nicht herunterladen {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"Fertig - gespeichert {url} - als {file_path}")
    except Exception as e:
        print(f"ERROR - Speicherfehler {url} - {e}")

In [21]:
def search_and_download(search_term:str,driver_path:str,number_images:int,target_path='C:/Users/Smurf/Pictures'):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)
        

In [22]:
search_term = 'Mona Lisa'
number_images=5

search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH,
    number_images=number_images
)

<ipython-input-21-f9a670009594>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
<ipython-input-19-f0da10b09b22>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Gefunden: 100 Suchergebnisse. Extrahiere Links von 0:100


<ipython-input-19-f0da10b09b22>:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Gefundene Bilderlinks: 5, Fertig!
Fertig - gespeichert https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTujfO6pNNXGYl9EGQDQi2KRFssbmJY_0fqMw&usqp=CAU - als C:/Users/Smurf/Pictures\mona_lisa\7087cf0512.jpg
Fertig - gespeichert https://static.dw.com/image/57811475_303.jpg - als C:/Users/Smurf/Pictures\mona_lisa\66d742913f.jpg
Fertig - gespeichert https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ4ZOyHgWLxALNO9Af7OSa1V6s9UEpzkWWfbg&usqp=CAU - als C:/Users/Smurf/Pictures\mona_lisa\4b991e8a1c.jpg
Fertig - gespeichert https://cdn.prod.www.spiegel.de/images/2bf57351-315c-48c9-aa2b-db43a090912a_w948_r1.778_fpx50_fpy30.jpg - als C:/Users/Smurf/Pictures\mona_lisa\f2ce2a6f81.jpg
Fertig - gespeichert https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTxs-2amvo9Je3at_3jLqnUAsVv0gMKyrz5Rg&usqp=CAU - als C:/Users/Smurf/Pictures\mona_lisa\680ef8b92a.jpg
